In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataquest-frippe-problem/sample_submission.csv
/kaggle/input/dataquest-frippe-problem/test.csv
/kaggle/input/dataquest-frippe-problem/data.csv


In [2]:
data = pd.read_csv('/kaggle/input/dataquest-frippe-problem/data.csv')

In [14]:
data.head()

,ID,marka,naw3,9at3a,khochn,toul,3ordh,R,G,B,soum
0,0,Bey&Bey,mta3 a3res,kabbout,1.081667,22.4,57.0,98,18,21,49.2
1,1,HA,confy,sabbat,1.038333,46.4,54.0,128,23,27,180.9
2,2,Zen,confy,maryoul,1.021667,40.8,54.0,121,22,25,143.8
3,3,Bey&Bey,confy,maryoul,1.065000,24.8,56.0,101,18,21,54.4
4,4,Armani,classy,kabbout,1.041667,72.0,61.0,146,26,30,345.9


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48548 entries, 0 to 48547
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      48548 non-null  int64  
 1   marka   48548 non-null  object 
 2   naw3    48548 non-null  object 
 3   9at3a   48548 non-null  object 
 4   khochn  48548 non-null  float64
 5   toul    48548 non-null  float64
 6   3ordh   48548 non-null  float64
 7   R       48548 non-null  int64  
 8   G       48548 non-null  int64  
 9   B       48548 non-null  int64  
 10  soum    48548 non-null  float64
dtypes: float64(4), int64(4), object(3)
memory usage: 4.1+ MB


In [145]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
def clean(data):
    scaler = StandardScaler()
    label = LabelEncoder()
    data.toul = scaler.fit_transform(np.array(data.toul).reshape(-1,1))
    data['3ordh'] = scaler.fit_transform(np.array(data["3ordh"]).reshape(-1,1))
    data['khochn'] = scaler.fit_transform(np.array(data["khochn"]).reshape(-1,1))
    data.R = scaler.fit_transform(np.array(data.R).reshape(-1,1))
    data.G = scaler.fit_transform(np.array(data.G).reshape(-1,1))
    data.B = scaler.fit_transform(np.array(data.B).reshape(-1,1))
    data.marka = label.fit_transform(data.marka)
    data.naw3 = label.fit_transform(data.naw3)
    data['9at3a'] = label.fit_transform(data['9at3a'])
    X = data.drop(columns=['soum','ID'])
    Y = data.soum
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=42)
    return X_train, X_test, y_train, y_test

In [146]:
X_train, X_test, y_train, y_test = clean(data)

In [147]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46120 entries, 47493 to 15795
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   marka   46120 non-null  int64  
 1   naw3    46120 non-null  int64  
 2   9at3a   46120 non-null  int64  
 3   khochn  46120 non-null  float64
 4   toul    46120 non-null  float64
 5   3ordh   46120 non-null  float64
 6   R       46120 non-null  float64
 7   G       46120 non-null  float64
 8   B       46120 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 3.5 MB


In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import Adam
model = Sequential([
    Input(shape = (9,)),
    Dense(units = 512, activation='relu'),
    Dense(units = 256, activation='relu'),
    Dense(units = 128, activation='relu'),
    Dense(units = 128, activation='relu'),
    Dense(units = 64, activation='relu'),
    Dense(units = 32, activation='relu'),
    Dense(units = 16, activation='relu'),
    Dense(units = 1, activation='linear')
])
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.95

# Create a learning rate schedule using exponential decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=False)

# Create the Adam optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae']) 
model.fit(X_train,y_train,batch_size=128,validation_data=(X_test,y_test),epochs=70)

Epoch 1/70
304/304 [==============================] - 4s 9ms/step - loss: 51818.3242 - mae: 124.9101 - val_loss: 20730.6406 - val_mae: 86.9938
Epoch 2/70
 94/304 [========>.....................] - ETA: 1s - loss: 19371.8320 - mae: 81.9383

KeyboardInterrupt: 

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
model1 = XGBRegressor(n_estimators=1700, max_depth=8, eta= 0.007,booster='dart')
model1.fit(X_train,y_train)
predictions = model1.predict(X_test)
r2 = r2_score(y_test,predictions)
print(r2)

In [59]:
model

In [77]:
predictions = model.predict(X_test)
r2 = r2_score(y_test,predictions)
print(r2)

304/304 [==============================] - 1s 2ms/step
0.9800161623700591


In [149]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

def clean_onehot(data):
    scaler = StandardScaler()
    encoder = OneHotEncoder()
    
    # Scale numerical features
    data.toul = scaler.fit_transform(np.array(data.toul).reshape(-1,1))
    data['3ordh'] = scaler.fit_transform(np.array(data["3ordh"]).reshape(-1,1))
    data['khochn'] = scaler.fit_transform(np.array(data["khochn"]).reshape(-1,1))
    data.R = scaler.fit_transform(np.array(data.R).reshape(-1,1))
    data.G = scaler.fit_transform(np.array(data.G).reshape(-1,1))
    data.B = scaler.fit_transform(np.array(data.B).reshape(-1,1))
    
    # Encode categorical features with OneHotEncoder
    marka_encoded = encoder.fit_transform(np.array(data.marka).reshape(-1, 1))
    naw3_encoded = encoder.fit_transform(np.array(data.naw3).reshape(-1, 1))
    hat3a_encoded = encoder.fit_transform(np.array(data['9at3a']).reshape(-1, 1))
    
    # Convert sparse matrices to arrays and concatenate
    encoded_features = np.concatenate([marka_encoded.toarray(), naw3_encoded.toarray(), hat3a_encoded.toarray()], axis=1)
    
    # Create DataFrame for encoded features
    encoded_df = pd.DataFrame(encoded_features, columns=['marka_' + str(i) for i in range(marka_encoded.shape[1])] + 
                                                    ['naw3_' + str(i) for i in range(naw3_encoded.shape[1])] +
                                                    ['hat3a_' + str(i) for i in range(hat3a_encoded.shape[1])])
    
    # Concatenate numerical and encoded categorical features
    data_encoded = pd.concat([data.drop(columns=['marka', 'naw3', '9at3a']), encoded_df], axis=1)
    
    return data_encoded

In [150]:
data_e = clean_onehot(data)
X = data_e.drop(columns=['soum'])
Y = data_e.soum
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [144]:
def clean_test(data):
    scaler = StandardScaler()
    label = LabelEncoder()
    data.toul = scaler.fit_transform(np.array(data.toul).reshape(-1,1))
    data['3ordh'] = scaler.fit_transform(np.array(data["3ordh"]).reshape(-1,1))
    data['khochn'] = scaler.fit_transform(np.array(data["khochn"]).reshape(-1,1))
    data.R = scaler.fit_transform(np.array(data.R).reshape(-1,1))
    data.G = scaler.fit_transform(np.array(data.G).reshape(-1,1))
    data.B = scaler.fit_transform(np.array(data.B).reshape(-1,1))
    data.marka = label.fit_transform(data.marka)
    data.naw3 = label.fit_transform(data.naw3)
    data['9at3a'] = label.fit_transform(data['9at3a'])
    return data

In [179]:
test = pd.read_csv('/kaggle/input/dataquest-frippe-problem/test.csv')

In [188]:
test_clean = clean_onehot(test)

In [190]:
predictions = model1.predict(test_clean.drop(columns=['ID']))

In [191]:
predictions_df = pd.DataFrame({'ID':test_clean.ID , 'Predictions': predictions})

In [192]:
predictions_df

,ID,Predictions
0,0,96.368706
1,1,228.078690
2,2,376.805969
3,3,61.107487
4,4,370.702881
...,...,...
5390,5390,735.109253
5391,5391,861.446350
5392,5392,109.253494
5393,5393,1351.632690


In [193]:
predictions_df.to_csv('submission3.csv',index=False)

In [159]:
data_enc = clean_onehot(data)

In [160]:
X = data_enc.drop(columns=['ID','soum'])
Y = data_enc.soum
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=42)